In [18]:
#Definir nuestro archivo a importar
filename = "Transacción de muestra.txt"

data = open(filename, "r", encoding = "utf-8")
read_data = data.read()
print(read_data)

************************ DATOS DE LA TRANSACCIÓN ***************************
Empresa: EjemploCorp
Fecha del Reporte: 2024-02-03
Número de Sucursal: 12345

ID_Transaccion, Fecha, Producto, Cantidad, Precio, Categoria, ID_Cliente, Notas
1, 2024-01-20, ProductoA, 5, 10.50, Electrónica, 101, -
2, 2024-01-21, ProductoB, 2, 15.75, Ropa, 102, -
3, 2024-01-22, ProductoC, 8, 7.20, Hogar, 103, -
4, 2024-01-23, ProductoA, 3, 10.50, Electrónica, 104, -
5, 2024-01-23, ProductoB, 1, 15.75, Ropa, 105, -
6, 2024-01-24, ProductoC, 4, 7.20, Hogar, 106, -
7, 2024-01-25, ProductoA, 6, 10.50, Electrónica, 107, -
8, 2024-01-25, ProductoB, 3, 15.75, Ropa, 108, -
9, 2024-01-26, ProductoC, 2, 7.20, Hogar, 109, -
10, 2024-01-27, ProductoA, 4, 10.50, Electrónica, 110, -
11, 2024-01-27, ProductoB, 5, 15.75, Ropa, 111, -
12, 2024-01-28, ProductoC, 7, 7.20, Hogar, 112, -
13, 2024-01-29, ProductoA, 2, 10.50, Electrónica, 113, -
14, 2024-01-29, ProductoB, 1, 15.75, Ropa, 114, -
15, 2024-01-30, ProductoC, 3, 7.20, Hog

In [19]:
import pandas as pd

def get_transactions_data(strData):
    data_array = []
    isTransaction = False
    for line in strData.split("\n"):
        if "ID_Transaccion" in line or isTransaction == True:
            isTransaction = True
            data_line = line.split(", ")
            data_array.append(data_line)
    df = pd.DataFrame(data_array)
    df.columns = df.iloc[0]
    df = df.drop([0])
    df = df.dropna()
    df = df.reset_index(drop=True)
    return df

In [20]:
def get_value_by_keyword(strData, strKeyword):
    for line in strData.split("\n"):
        if strKeyword in line:
            data_value = line.split(": ")
            value = data_value[1]
            break
    return value
    

In [21]:
def preliminar_data(strData):
    out_dict = {"Empresa": "", 
                "Fecha": "",
                "Sucursal": "",
                "Transacciones": None
               }
    out_dict["Empresa"] = get_value_by_keyword(strData, "Empresa")
    out_dict["Fecha"] = get_value_by_keyword(strData, "Fecha del Reporte")
    out_dict["Sucursal"] = get_value_by_keyword(strData, "Número de Sucursal")
    out_dict["Transacciones"] = get_transactions_data(strData)
    return out_dict




In [22]:
#Realizar las Siguientes tareas: 
#1) Cambiar el nombre de la columna Precio a Precio Unitario
#2)Crear una nueva columna llamada total de ventas que sea el total de la cantidad * Precio
#3)Obtener el total de venta x cada categoria
#4)Crear las columnas mes dia y anho
#5)Obtener el producto y fecha con mas venta
 
#6)Graficar el historico de ventas x dia 
#7)Graficar el numero de productos vendidos x dia 
#8)Exportar todo a un resumen ejecutivo en excel

In [23]:
def get_data_analysis(df_input):
    df = df_input.copy()
    df = df.rename(columns={"Precio" : 'Precio Unitario'})#1
    df["Total de Ventas"]= df["Cantidad"].astype(float) * df["Precio Unitario"].astype(float)#2
    df_sales_category  = df.groupby("Categoria").sum("Total de Ventas")#3
    df["Fecha"]= pd.to_datetime(df["Fecha"], yearfirst=True)#4
    df["Mes"] = df["Fecha"].dt.month
    df["Dia"] = df["Fecha"].dt.day
    df["Ano"] = df["Fecha"].dt.year
    most_sold_index = df["Total de Ventas"].idxmax()
    most_sold_product = df.iloc[most_sold_index]["Producto"]
    most_sold_date = df.iloc[most_sold_index]["Fecha"]
    
    out_dict = {"data": df, 
                "Sales_by_category": df_sales_category,
                "Most _sold_info: ": {"Product:" : most_sold_product, "Date:": most_sold_date}}
    
    return out_dict





Graficos

In [24]:
from matplotlib import pyplot as plt
import matplotlib.ticker as FuncFormatter

def plot_sales_day(dfi, figname):
    df = dfi.copy()
    df = df[["Fecha", "Total de Ventas"]].groupby('Fecha').sum()
    df = df.reset_index()
    plt.figure(figsize=(15,5))
    plt.plot(df["Fecha"], df["Total de Ventas"], marker='o', color="green")
    

    plt.grid(True)
    plt.xticks(df["Fecha"], rotation=45)
    
    
    plt.savefig(figname)
    plt.close()
    
    return    



In [25]:
def sales_by_category(dfi, figname):
    df = dfi.copy()
    df = df.reset_index()
    
    plt.title("Ventas por categoria")
    
    plt.pie(df["Total de Ventas"], labels = df["Categoria"], colors=["skyblue", "lightgreen", "lightcoral"])
    plt.legend(title = "Categoria")
    plt.savefig(figname)
    plt.close()
    return 



In [26]:
#Graficar los productos de menor a mayor ventas

def sales_by_product(dfi, figname):
    df = dfi.copy()
    df = df[["Producto", "Total de Ventas"]].groupby('Producto').sum()
    df = df.reset_index()
    df = df.sort_values(by="Total de Ventas", ascending=True)
    
    plt.bar(df["Producto"], df["Total de Ventas"], width=0.6, color="#6F00FF")
    plt.xlabel("Nombre del Producto")
    plt.ylabel("Total de Ventas")   

    plt.savefig(figname)
    plt.close()
    
    return
    


Reporte Excel


In [27]:
import openpyxl
from openpyxl import workbook 

def reporte_xls(txt_filename):
    data = open(txt_filename, "r", encoding = "utf-8")
    txt_data = data.read()
    preliminar = preliminar_data(read_data)
    #Analisis de Datos 
    analysis = get_data_analysis(preliminar["Transacciones"])
    #Graficos
    sales_by_product(analysis["data"], "./img/Ventas por producto.png")
    plot_sales_day(analysis["data"], "./img/Historico Ventas.png")
    sales_by_category(analysis["Sales_by_category"], "./img/Ventas por Categoria.png")
    #Reporte excel
    wb = workbook()
    ws = wb.worksheets[0]
    
    
    return

reporte_xls("Transacción de muestra.txt")

TypeError: 'module' object is not callable